In [ ]:
!pip install PyPDF2
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._dete

In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
model = "tiiuae/falcon-7b-instruct"

In [3]:
PATH = "pdf/JavaBasico.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")

In [4]:
text_split = []
metadata=[]
chunk_size = 1024
chunk_overlap = 8

for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})

In [5]:
embeddings_db = embeddings.encode(text_split)

In [6]:
for i, d in enumerate(text_split):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadata[i]
    )

In [7]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
prompt = "o que é arroz?"

prompt_embeddings = embeddings.encode([prompt])

results = collection.query(
    query_embeddings=prompt_embeddings,
    n_results=3,
    where={"name": "book-langchain"}
    )

distance = results['distances']
print(distance)

data = results['documents'][0]
print(data)

[[0.9344969391822815, 1.0258688926696777, 1.0394511222839355]]
['io  para fazer a leitura de um arquivo que está num determinado \ndiretório; assim, você deve esperar, por exemplo, um erro tal que o arquivo não possa ser encontrado, o que é um tipo de IOException , do \ntipo FileNotFoundException , que não é ﬁ  lha de RunTimeException . \nÉ, assim, uma Checked Exception , e você é obrigado a tratá-la, caso \ncontrário a classe não irá nem compilar, lançando qual exceção levou ao erro: unreported exception java.io.FileNotFoundException ; must be \ncaught or declared to be thrown (erro apresentado pelo compilador). \nVeja como é fácil perceber qual exceção deixou de ser tratada: o \ncompilador mostra na sua janela de erro (com o tempo e a experiência, você aprenderá sobre elas).\nOutro detalhe que merece atenção: já que uma classe pode fazer \nvárias coisas, podemos ter mais de uma exceção e de repente todas checked .\nAssim, podemos encadear tratamentos de exceção, mas cada um \nem um ú

In [11]:
torch.manual_seed(4)

prompt = f"""Answer the question in portuguese using only the context below.

Context: {data}

Question: {prompt}

Answer:

"""

sequences = pipe(
    prompt,
    max_new_tokens=512,
    do_sample=True, #introduz um elemento de aleatoriedade, injetando criatividade na produção do LLM
    top_k=10, #número de escolhas de vocabulário de maior probabilidade
    return_full_text = False,
    temperature=0.1
)

for seq in sequences:

    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: <p>Arroz é um tipo de arroba, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma espécie de arroba de arroz, que é uma es